In [13]:
import torch
import torch.nn as nn

# IMPORTING NECESSARY LIBRARIES
# TORCH IS THE MAIN PYTORCH LIBRARY FOR DEEP LEARNING
# NN IS A SUBMODULE OF TORCH THAT CONTAINS NEURAL NETWORK LAYERS AND FUNCTIONS
# THESE IMPORTS ARE ESSENTIAL FOR BUILDING AND TRAINING NEURAL NETWORKS

class Discriminator(nn.Module):
    
    # DEFINING THE DISCRIMINATOR CLASS WHICH INHERITS FROM NN.MODULE
    # THIS CLASS REPRESENTS THE DISCRIMINATOR PART OF THE DCGAN
    # INHERITANCE FROM NN.MODULE ALLOWS US TO USE PYTORCH'S BUILT-IN FUNCTIONALITIES

    def __init__(self, channels_img, features_d):
        # CONSTRUCTOR METHOD FOR THE DISCRIMINATOR
        # CHANNELS_IMG: NUMBER OF INPUT IMAGE CHANNELS (E.G., 3 FOR RGB)
        # FEATURES_D: BASE NUMBER OF FEATURES IN THE DISCRIMINATOR
        super(Discriminator, self).__init__()
        # CALLING THE PARENT CLASS CONSTRUCTOR
        # THIS IS NECESSARY WHEN INHERITING FROM NN.MODULE

        self.disc = nn.Sequential(
            # CREATING A SEQUENTIAL CONTAINER FOR THE DISCRIMINATOR LAYERS
            # THIS ALLOWS US TO STACK LAYERS IN A SPECIFIC ORDER

            # INPUT N X CHANNELS_IMG X 64 X 64
            nn.Conv2d(channels_img, features_d, kernel_size=4, stride=2, padding=1),  # 32 X 32
            # FIRST CONVOLUTIONAL LAYER
            # REDUCES SPATIAL DIMENSIONS AND INCREASES FEATURE DEPTH
            # KERNEL_SIZE=4, STRIDE=2, AND PADDING=1 ARE TYPICAL FOR DCGAN

            nn.LeakyReLU(0.2),
            # ACTIVATION FUNCTION
            # LEAKYRELU ALLOWS SMALL NEGATIVE VALUES, HELPING WITH GRADIENT FLOW

            self._block(features_d, features_d*2, 4, 2, 1),  # 16 X 16
            self._block(features_d*2, features_d*4, 4, 2, 1),  # 8 X 8
            self._block(features_d*4, features_d*8, 4, 2, 1),  # 4 X 4
            # CALLING THE _BLOCK METHOD TO CREATE ADDITIONAL CONVOLUTIONAL BLOCKS
            # EACH BLOCK FURTHER REDUCES SPATIAL DIMENSIONS AND INCREASES FEATURE DEPTH

            nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=2, padding=0),  # 1 X 1
            # FINAL CONVOLUTIONAL LAYER
            # REDUCES THE OUTPUT TO A SINGLE VALUE PER IMAGE

            nn.Sigmoid()
            # FINAL ACTIVATION FUNCTION
            # SQUASHES THE OUTPUT BETWEEN 0 AND 1, REPRESENTING PROBABILITY OF REAL VS FAKE
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        # HELPER METHOD TO CREATE A CONVOLUTIONAL BLOCK
        # THIS REDUCES CODE REPETITION AND IMPROVES READABILITY
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
            # CONVOLUTIONAL LAYER
            # BIAS=FALSE BECAUSE BATCHNORM WILL HANDLE THE BIAS

            nn.BatchNorm2d(out_channels),
            # BATCH NORMALIZATION
            # HELPS WITH TRAINING STABILITY AND ALLOWS HIGHER LEARNING RATES

            nn.LeakyReLU(0.2),
            # ACTIVATION FUNCTION
            # CONSISTENT USE OF LEAKYRELU THROUGHOUT THE NETWORK
        )

    def forward(self, x):
        # FORWARD METHOD DEFINES HOW INPUT DATA FLOWS THROUGH THE NETWORK
        return self.disc(x)
        # PASSES THE INPUT THROUGH THE ENTIRE DISCRIMINATOR NETWORK

In [14]:
class Generator(nn.Module):
    # DEFINES THE GENERATOR CLASS WHICH INHERITS FROM PYTORCH'S NN.MODULE
    # THIS CLASS WILL BE RESPONSIBLE FOR GENERATING FAKE IMAGES

    def __init__(self, z_dim, channels_img, features_g):
        # CONSTRUCTOR METHOD THAT INITIALIZES THE GENERATOR
        # Z_DIM: DIMENSION OF THE RANDOM NOISE INPUT
        # CHANNELS_IMG: NUMBER OF COLOR CHANNELS IN THE OUTPUT IMAGE (E.G., 3 FOR RGB)
        # FEATURES_G: BASE NUMBER OF FEATURES (FILTERS) IN THE GENERATOR

        super(Generator, self).__init__()
        # CALLS THE CONSTRUCTOR OF THE PARENT CLASS (NN.MODULE)

        self.net = nn.Sequential(
            # DEFINES THE NETWORK ARCHITECTURE AS A SEQUENCE OF LAYERS

            self._block(z_dim, features_g * 16, 4, 1, 0),
            # FIRST BLOCK: EXPANDS THE INPUT NOISE TO FEATURES_G * 16 CHANNELS
            # ADVANTAGES: HELPS IN UPSCALING THE INPUT AND CREATING INITIAL FEATURES

            self._block(features_g * 16, features_g * 8, 4, 2, 1),
            self._block(features_g * 8, features_g * 4, 4, 2, 1),
            self._block(features_g * 4, features_g * 2, 4, 2, 1),
            # SUBSEQUENT BLOCKS: GRADUALLY REDUCE THE NUMBER OF CHANNELS WHILE INCREASING SPATIAL DIMENSIONS
            # ADVANTAGES: ALLOWS THE NETWORK TO LEARN HIERARCHICAL FEATURES AND UPSCALE THE IMAGE

            nn.ConvTranspose2d(
                features_g * 2, channels_img, 4, 2, 1
            ),
            # FINAL TRANSPOSED CONVOLUTION: PRODUCES THE OUTPUT IMAGE WITH THE DESIRED NUMBER OF CHANNELS
            # ADVANTAGES: CREATES THE FINAL IMAGE OUTPUT WITH THE CORRECT DIMENSIONS AND CHANNEL COUNT

            nn.Tanh()
            # ACTIVATION FUNCTION: SQUASHES THE OUTPUT TO THE RANGE [-1, 1]
            # ADVANTAGES: HELPS IN PRODUCING NORMALIZED IMAGE PIXEL VALUES
        )    

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        # HELPER METHOD TO CREATE A BLOCK OF LAYERS
        # THIS METHOD IS USED TO AVOID REPETITION IN THE MAIN ARCHITECTURE

        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
            # TRANSPOSED CONVOLUTION: UPSCALES THE INPUT WHILE CHANGING THE NUMBER OF CHANNELS
            # ADVANTAGES: ALLOWS FOR LEARNABLE UPSCALING OF FEATURES

            nn.BatchNorm2d(out_channels),
            # BATCH NORMALIZATION: NORMALIZES THE OUTPUT OF THE PREVIOUS LAYER
            # ADVANTAGES: HELPS IN STABILIZING TRAINING AND ALLOWS HIGHER LEARNING RATES

            nn.ReLU()
            # ACTIVATION FUNCTION: INTRODUCES NON-LINEARITY
            # ADVANTAGES: HELPS THE NETWORK LEARN COMPLEX PATTERNS AND FEATURES
        )
    
    def forward(self, x):
        # FORWARD METHOD DEFINES HOW INPUT DATA FLOWS THROUGH THE NETWORK
        return self.net(x)
        # PASSES THE INPUT THROUGH THE ENTIRE GENERATOR NETWORK
        

In [15]:
def intialize_weights(model):
    # HELPER FUNCTION TO INITIALIZE THE WEIGHTS OF THE NETWORK
    # THIS FUNCTION WILL BE USED TO INITIALIZE THE WEIGHTS OF THE DISCRIMINATOR AND GENERATOR

    for m in model.modules():
        # ITERATES OVER ALL MODULES IN THE NETWORK

        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            # CHECKS IF THE MODULE IS A CONVOLUTIONAL LAYER OR BATCH NORMALIZATION LAYER

            nn.init.normal_(m.weight.data, 0.0, 0.02)
            # INITIALIZES THE WEIGHTS OF THE LAYER USING A NORMAL DISTRIBUTION
            # MEAN=0.0, STD=0.02           

In [16]:
def test():
    # DEFINE TEST PARAMETERS
    N, in_channels, H, W = 8, 3, 64, 64
    # N: BATCH SIZE, USED TO TEST MULTIPLE INPUTS AT ONCE
    # in_channels: NUMBER OF INPUT CHANNELS (3 FOR RGB IMAGES)
    # H, W: HEIGHT AND WIDTH OF THE INPUT IMAGES

    z_dim = 100
    # DIMENSION OF THE LATENT SPACE VECTOR FOR THE GENERATOR

    # CREATE A RANDOM TENSOR TO SIMULATE INPUT IMAGES
    x = torch.randn((N, in_channels, H, W))
    # ADVANTAGES: ALLOWS TESTING WITH VARIED INPUTS, SIMULATES REAL-WORLD SCENARIOS

    # INSTANTIATE THE DISCRIMINATOR
    disc = Discriminator(in_channels, 8)
    # 8: STARTING FEATURE MAP SIZE, WILL BE DOUBLED IN EACH LAYER
    # ADVANTAGES: MODULAR DESIGN, EASY TO ADJUST NETWORK COMPLEXITY

    # INITIALIZE WEIGHTS OF THE DISCRIMINATOR
    intialize_weights(disc)
    # ENSURES PROPER INITIALIZATION FOR BETTER TRAINING CONVERGENCE

    # TEST DISCRIMINATOR OUTPUT SHAPE
    assert disc(x).shape == (N, 1, 1, 1), "Discriminator test failed"
    # ENSURES THE DISCRIMINATOR OUTPUTS A SINGLE VALUE FOR EACH INPUT IMAGE
    # ADVANTAGES: CATCHES POTENTIAL ERRORS IN NETWORK ARCHITECTURE

    # INSTANTIATE THE GENERATOR
    gen = Generator(z_dim, in_channels, 8)
    # ADVANTAGES: MODULAR DESIGN, CONSISTENT WITH DISCRIMINATOR ARCHITECTURE

    # INITIALIZE WEIGHTS OF THE GENERATOR
    intialize_weights(gen)
    # ENSURES PROPER INITIALIZATION FOR BETTER TRAINING CONVERGENCE

    # CREATE A RANDOM LATENT VECTOR
    z = torch.randn((N, z_dim, 1, 1))
    # INPUT FOR THE GENERATOR, SIMULATES RANDOM NOISE

    # TEST GENERATOR OUTPUT SHAPE
    assert gen(z).shape == (N, in_channels, H, W), "Generator test failed"
    # ENSURES THE GENERATOR PRODUCES IMAGES OF THE CORRECT SIZE AND CHANNEL COUNT
    # ADVANTAGES: CATCHES POTENTIAL ERRORS IN NETWORK ARCHITECTURE

    print("Success")
    # INDICATES ALL TESTS PASSED SUCCESSFULLY

# RUN THE TEST FUNCTION
test()
# ADVANTAGES: ENCAPSULATES ALL TESTS IN A SINGLE FUNCTION, EASY TO RUN AND EXTEND

Success
